In [17]:
#IMPORTING DEPENDENCIES

from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [3]:
# Scrape NASA Mars News
browser = init_browser()

# Visit NASA Mars News site
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

time.sleep(1)

# Instantiate BS object
html = browser.html
soup = bs(html, "html.parser")

# Find news slide
news_slide = soup.find('li', class_='slide')

# Scrape slide title
news_title = news_slide.find('div', class_='content_title').get_text()

# Scrape slide text
news_preview  = news_slide.find('div', class_='article_teaser_body').get_text()

# Store data in a dictionary
news_data = {
    
    "title": news_title,
    "content": news_preview
}

# Close the browser after scraping
browser.quit()

In [15]:
# Scrape JPL Mars Featured Images
browser = init_browser()

# Visit JPL site
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

time.sleep(1)

# Instantiate BS object
html = browser.html
soup = bs(html, "html.parser")

#browser.links.find_by_partial_text('FULL IMAGE').click()

# Find featured image link
img_url = soup('article', class_='carousel_item')[0]['style'][23:-3]
jpl_url = "https://www.jpl.nasa.gov" + img_url

# Close the browser after scraping
browser.quit()

In [29]:
# Use Pandas html function to scrape Space Facts table
df = pd.read_html(url)[0]

# Convert dataframe back into html
page_table = df.to_html()

In [69]:
# Scrape Astrogeology Images
browser = init_browser()

# Visit NASA Mars News site
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

time.sleep(1)

# Instantiate BS object
html = browser.html
soup = bs(html, "html.parser")

#browser.links.find_by_partial_text('FULL IMAGE').click()

# Find page link containers
link_list_raw = soup.find_all('a', class_='itemLink product-item')

# Extract page urls and concatenate with base url for Splinter
link_list = []
base_url = "https://astrogeology.usgs.gov"
for item in link_list_raw:
    if item['href'] not in link_list:
        link_list.append(item['href'])

searches = [base_url + item for item in link_list]

# Close the browser after scraping
browser.quit()

In [80]:
# Splinter search all item pages
browser = init_browser()

# Create list of dictionaries
dictionary_list = []
for link in searches:
    dictionary = {}
    browser.visit(link)
    html = browser.html
    soup = bs(html, "html.parser")
    img_url = soup.find('img', class_='wide-image')['src']
    raw_title = soup.find('h2', class_='title').text.split()
    title = " ".join(raw_title[:-1])
    dictionary['title'] = title
    dictionary['img_url'] = base_url + img_url
    dictionary_list.append(dictionary)
    browser.back()
    
# Close the browser after scraping
browser.quit()

In [81]:
dictionary_list

[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]